<img src="./images/title.png"/>




<img src="./images/prob_1/p1.png"/>

<img src="./images/prob_1/a.png"/>

Dual problem presented in problem 1 :

minimize   $12\pi_{1} + 8\pi_{2}$

subject to : 

$x_{1}: 4\pi_{1} + \pi_{2} \geq 1 \\
x_{2}: \pi_{1} + 2\pi_{2} \geq 2 \\
x_{3}: 3\pi_{1} - 2\pi_{2} \geq -8$

### Feasable region plot
<img src="./images/fig1.png" style="width:460px;"/>

Figure 1


### Feasable solutions
<img src="./images/fig2.png" style="width:460px;"/>

Figure 2


As we can see in Figure 2, all feasable solutions would lie in the feasable region's edge $x_{2}: \pi_{1} + 2\pi_{2} = 2$. Such line segment has vertex $(0,1)$, which is optimal and unique.

$(\pi_{1}^{*},\pi_{2}^{*}) = (0,1)$



<img src="./images/prob_1/b.png"/>

#### Optimality condition 1: 
$\bf{x^{*}}$ is primal feasable as it satisfies the inequalities in the primal problem

#### Optimality condition 2:
$\bf{\pi^{*}}$ is dual feasable as it satisfies the inequalities in the dual problem

#### Optimality condition 3:
Complementary slackness:

$x_{3}: 3\pi_{1} - 2\pi_{2} \geq -8$ always holds

- therefore: $x_{3} = 0$

Then we have:

$4x_{1} + x_{2} = 12 \\
x_{1} + 2x_{2} = 8 \\
x_{1} \geq 0, x_{2} \geq 0$

Solving for $x_{1}$ and $x_{2}$:

$x_{2} = 12 - 4x_{1} \\
x_{1} - 8x_{1} + 24 = 8 \\
-7x_{1} = -16$

Which results in:

$x_{1} = \frac{16}{7} \\
x_{2} = \frac{20}{7}$

 - therefore ${\bf x^{*}} = (\frac{16}{7}, \frac{20}{7}, 0)$




### Solving the problem in Gurobi-Python

In [101]:
# importing libraries
import gurobipy as gp
from gurobipy import GRB

# setting up the optimization problem
m = 2  # number of resources
n = 3  # number of products

resources = range(m)  # list [1, ..., m]
products = range(n)  # list [1, ..., n]

# primal objective coefficients
r_coeff = [1, 2, -8]

# left-hand side (LHS) coefficients (matrix A)
A_coeff = [[4, 1, 3],
           [1, 2, -2]]

# right-hand side (RHS) coefficients
b_coeff = [12, 8]

r = {j: r_coeff[j] for j in products}
A = {i: {j: A_coeff[i][j] for j in products} for i in resources}
b = {i: b_coeff[i] for i in resources}

# naming the model
model = gp.Model('1d')
x = model.addVars(products, name="x")  # quantity produced

# Constraints
model.addConstrs((gp.quicksum(A[i][j] * x[j] for j in products)
                  == b[i]
                  for i in resources), name="pi")

# Objective
obj = gp.quicksum(r[j] * x[j] for j in products)
model.setObjective(obj, GRB.MAXIMIZE)
model.optimize()


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xd85cf6b9
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 1e+01]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   2.750000e+30   3.000000e+00      0s
       3    8.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds
Optimal objective  8.000000000e+00


In [102]:
# Printing solution
# Variable information including sensitivity information
varnames = [f"x[{n+1}]" for n,x in enumerate(model.getVars())]
for var, v in zip(varnames,model.getVars()):
    print(f"{var} = {v.X}")

x[1] = 2.2857142857142856
x[2] = 2.857142857142857
x[3] = 0.0


<img src="./images/prob_1/c.png"/>

The reduced costs of the primal variables are computed as following:

$\bar{c_{1}} = 4\pi_{1}^{*} + \pi_{2}^{*} - 1 = 4\lambda + 1 - \frac{\lambda}{2} - 1 = \frac{7\lambda}{2}  \\
\bar{c_{2}} = \pi_{1}^{*} + 2\pi_{2}^{*} - 2 = \lambda + 2 - \lambda - 2 = 0 \\
\bar{c_{3}} = 3\pi_{1}^{*} - 2\pi_{2}^{*} + 8 = 3\lambda - 2 - \lambda + 8 = 2\lambda + 6$

Where $\bf{\pi^{*}}$ is a dual optimal solution
 - therefore, the reduced costs are not unique

<img src="./images/prob_1/d.png"/>

primal problem:

maximize $x_{1} + 2x_{2} - 8x_{3}$

subject to :

$4x_{1} + x_{2} + 3x_{3} = 12 \\
x_{1} + 2x_{2} - 2x_{3} = 8 \\
x_{1} \geq 0, x_{2} \geq 0, x_{3} \geq 0$

Let's consider $\bf{\pi^{*}} = (0,1)$ we have $x_{3} = 0$ and then:

$4x_{1} + x_{2} = 12 + \Delta b_{1} \\
x_{1} + 2x_{2} = 8 + \Delta b_{2}$

Solving we get:

$\hat{x}^{*} = (\frac{1}{7} (2\Delta b_{1} - \Delta b_{2} + 16), \frac{1}{7} (-\Delta b_{1} + 4\Delta b_{2} + 20), 0)$

Imposing primal feasability returns:

$2\Delta b_{1} - \Delta b_{2} \geq -16 \\
-\Delta b_{1} + 4\Delta b_{2} \geq -20$

As a result: if $\Delta b_{1} = 0$ then $\Delta b_{2} \in [-5, 16]$ and if $\Delta b_{2} = 0$ then $\Delta b_{1} \in [-8, 20]$



### Comparing with Gurobi-Python

In [103]:
[x.X for x in model.getVars()]

[2.2857142857142856, 2.857142857142857, 0.0]

In [104]:
# Changes in constraint RHS
constnames = [(f"pi[{n+1}]",diff) for n,(x,diff) in enumerate(zip(model.getConstrs(),[12,8]))] 
for (cname,diff),c in zip(constnames,model.getConstrs()):
    print(f"{cname} : from RHS = {c.SARHSLow-diff}, to RHS = {c.SARHSUp-diff}")

pi[1] : from RHS = -8.0, to RHS = 20.0
pi[2] : from RHS = -5.0, to RHS = 16.0


As we can see, te Gurobi-Python solution matches both the intervals resulting from the analytical process.

<img src="./images/prob_1/e.png"/>

The primal problem only has one feasable solution, so the optimal solution $\bf{x^{*}}$ remains optimal for the following modified problem:

maximize $(1 + \Delta r_{1})x_{1} + (2 + \Delta r_{2})x_{2} - (8 + \Delta r_{3})x_{3}$

subject to :

$4x_{1} + x_{2} + 3x_{3} = 12 \\
x_{1} + 2x_{2} - 2x_{3} = 8 \\
x_{1} \geq 0, x_{2} \geq 0, x_{3} \geq 0$

Following a standard analysis, we consider ${\bf\hat{x}^{*}} = (\frac{16}{7}, \frac{20}{7}, 0)$

By complementary slackness we formulate the following 2 systems of linear equations:

#### system 1:

$4\pi_{1} + \pi_{2} = 1 + \Delta r_{1} \\
\pi_{1} + 2\pi_{2} = 2 + \Delta r_{2}$

resulting in: 

$\hat{\pi}^{*} = (\frac{1}{7} (2\Delta r_{1} - \Delta r_{2}), \frac{1}{7} (7 - \Delta r_{1} + 4\Delta b_{2}))$

Dual feasability means that:


$3\hat{\pi}^{*}_{1} + 2\hat{\pi}^{*}_{2} + 8 - \Delta r_{3} = \frac{1}{7} (42 + 8\Delta r_{1} - 11\Delta r_{2} - 7\Delta r_{3})$

and:

$8\Delta r_{1} - 11\Delta r_{2} - 7\Delta r_{3} \geq -42$

So, if:

$\Delta r_{1} = 0, \Delta r_{2} = 0$, then $\Delta r_{3} \leq 6$

$\Delta r_{1} = 0, \Delta r_{3} = 0$, then $\Delta r_{2} \leq \frac{42}{11}$

$\Delta r_{2} = 0, \Delta r_{3} = 0$, then $\Delta r_{1} \geq \frac{-21}{4}$

#### system 2:

$\pi_{1} + 2\pi_{2} = 2 + \Delta r_{2} \\
3\pi_{1} - 2\pi_{2} = -8 + \Delta r_{3}$

resulting in: 

$\hat{\pi}^{*} = (\frac{1}{4} (- 14 - 3\Delta r_{2} + \Delta r_{3}), \frac{1}{4} (- 20 -2\frac{1}{2}\Delta r_{2} + 2\frac{1}{2}\Delta r_{3}))$

Dual feasability means that:


$4\hat{\pi}^{*}_{1} + \hat{\pi}^{*}_{2} -1 - \Delta r_{1} = \frac{1}{4} (- 98 - 4\Delta r_{1} - 11\Delta r_{2} + 9\Delta r_{3})$

and:

$-4\Delta r_{1} - 11\Delta r_{2} + 9\Delta r_{3} \geq 98$

So, if:

$\Delta r_{1} = 0, \Delta r_{2} = 0$, then $\Delta r_{3} \geq \frac{98}{9}$

$\Delta r_{1} = 0, \Delta r_{3} = 0$, then $\Delta r_{2} \leq \frac{98}{11}$

$\Delta r_{2} = 0, \Delta r_{3} = 0$, then $\Delta r_{1} \leq \frac{-49}{2}$

Combining both cases we obtain that $\bf x^{*}$ is optimal for any $\Delta r_{j} \in \mathbb{R}$

### Comparing with Gurobi-Python

In [100]:
# Changes of objective coefficients for the primal problem
varnames = [(f"pi[{n+1}]",diff) for n,(x,diff) in enumerate(zip(model.getVars(),[1,2,-8]))]
for (var,diff), v in zip(varnames,model.getVars()):
    print(f"{var} : from coeff = {v.SAObjLow - diff}, to coeff = {v.SAObjUp - diff}")

pi[1] : from coeff = -5.25, to coeff = inf
pi[2] : from coeff = -inf, to coeff = 3.8181818181818183
pi[3] : from coeff = -inf, to coeff = 6.0


As we can see, the Gurobi-Python solution matches the first 3 differences 

<img src="./images/prob_2/p2.png"/>

<img src="./images/prob_2/a.png"/>

The primal LO formulation for LR with MAE criterion as seen in class has the following parameters:

- Objective: minimize $\sum_{i=1}^{n} (e_{i}^{+} + e_{i}^{-})$

- Subject to the following constraints: 
    - $(e_{i}^{+} - e_{i}^{-}) + b_{0} + x_{i1} b_{1} + \dots + x_{im} b_{m} = y_{i}$ $i = 1, \dots, n$

    - $e_{i}^{+}, e_{i}^{-} \geq 0,$ $i = 1, \dots, n$

### Gurobi implementation of the problem

In [12]:
# importing libraries
import gurobipy as gp
from gurobipy import GRB

In [13]:
# creating multidict with observations
observations, x, y = gp.multidict({
    ('1'): [29.7,175.3],
    ('2'): [29.5,177.8],
    ('3'): [31.4,185.4],
    ('4'): [31.8,175.3],
    ('5'): [27.6,172.7],
})

In [14]:
model = gp.Model('CurveFitting')

# Constant term of the function f(x). This is a free continuous variable that can take positive and negative values. 
a = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="a")

# Coefficient of the linear term of the function f(x). This is a free continuous variable that can take positive 
# and negative values.
b = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="b")

# Non-negative continuous variables that capture the positive deviations
e_plus = model.addVars(observations, vtype=GRB.CONTINUOUS, name="e_plus")

# Non-negative continuous variables that capture the negative deviations
e_neg = model.addVars(observations, vtype=GRB.CONTINUOUS, name="e_neg")

# Non-negative continuous variables that capture the value of the maximum deviation
z = model.addVar(vtype=GRB.CONTINUOUS, name="z")

In [15]:
# Deviation constraints

deviations = model.addConstrs( (b*x[i] + a + e_plus[i] - e_neg[i] == y[i] for i in observations), name='deviations')

# Objective function of problem 1

model.setObjective(e_plus.sum('*') + e_neg.sum('*'))

# Verify model formulation

model.write('CurveFitting.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 5 rows, 13 columns and 20 nonzeros
Model fingerprint: 0x72b80cf1
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 5 rows, 12 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
       5    1.3342857e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds
Optimal objective  1.334285714e+01


In [16]:
# The best straight line that minimizes the absolute value of the deviations is:
print(f"y = {b.x:.4f}x + ({a.x:.4f})")

y = 1.2381x + (138.5286)


In [17]:
# # Maximum deviation constraints

# maxPositive_deviation = model.addConstrs( (z >= e_plus[i] for i in observations), name='maxPositive_deviation')

# maxNegative_deviation = model.addConstrs( (z >= e_neg[i] for i in observations), name='maxNegative_deviation')

# # Objective function for Problem 2

# model.setObjective(z)

# # Run optimization engine

# model.optimize()

In [18]:
# # The best straight line that minimizes the maximum deviation is: 
# print(f"y = {b.x:.4f}x + ({a.x:.4f})")

In [19]:
for n,v in enumerate(model.getVars()):
    print(f"{v.Varname} = {v.X}, reduced cost = {abs(v.RC)}, from coeff = {v.SAObjLow}, to coeff = {v.SAObjUp}")

a = 138.52857142857118, reduced cost = 0.0, from coeff = -0.050505050505050386, to coeff = 0.021739130434782625
b = 1.2380952380952468, reduced cost = 0.0, from coeff = -0.6000000000000005, to coeff = 1.4999999999999967
e_plus[1] = 0.0, reduced cost = 1.7142857142857135, from coeff = -0.7142857142857135, to coeff = inf
e_plus[2] = 2.7476190476190467, reduced cost = 0.0, from coeff = -0.8947368421052615, to coeff = 1.315789473684211
e_plus[3] = 7.995238095238072, reduced cost = 0.0, from coeff = 0.11764705882353255, to coeff = 1.1578947368421055
e_plus[4] = 0.0, reduced cost = 2.0, from coeff = -1.0, to coeff = inf
e_plus[5] = 0.0, reduced cost = 1.2857142857142865, from coeff = -0.2857142857142865, to coeff = inf
e_neg[1] = 0.0, reduced cost = 0.28571428571428636, from coeff = 0.7142857142857136, to coeff = inf
e_neg[2] = 0.0, reduced cost = 2.0, from coeff = -1.0, to coeff = inf
e_neg[3] = 0.0, reduced cost = 2.0, from coeff = -1.0, to coeff = inf
e_neg[4] = 2.6000000000000227, reduce

In [21]:
# Optimal shadow prices
for n,c in enumerate(model.getConstrs()):
    print(f"{c.ConstrName} : shadow price = {c.Pi}, from RHS = {c.SARHSLow}, to RHS = {c.SARHSUp:.2f}, diff = {c.SARHSLow - c.SARHSUp}")

deviations[1] : shadow price = -0.7142857142857136, from RHS = 174.0, to RHS = 178.34, diff = -4.336842105263173
deviations[2] : shadow price = 1.0, from RHS = 175.05238095238096, to RHS = inf, diff = -inf
deviations[3] : shadow price = 1.0, from RHS = 177.40476190476193, to RHS = inf, diff = -inf
deviations[4] : shadow price = -1.0, from RHS = -inf, to RHS = 177.90, diff = -inf
deviations[5] : shadow price = -0.28571428571428636, from RHS = 162.82352941176472, to RHS = 175.30, diff = -12.476470588235287


<img src="./images/prob_2/b.png"/>

The prediction equation (representing the optimal solution) resulting from the implementation is $\hat{y} = 1.2549x + 138.5863$. 

In this case, the variables that have to take the value zero in any optimal solution because htey have positive reduced costs are:

$e_{1}^{+} = e_{2}^{+} = e_{4}^{+} = e_{5}^{+} = e_{6}^{+} = e_{3}^{-} = e_{7}^{-} = 0$ 

<img src="./images/prob_2/c.png"/>

So, we have the following system of equations that characterizest he optimal solutions:




<img src="./images/prob_3/p3.png"/>

<img src="./images/prob_3/a.png"/>

In [2]:
# importing libraries
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB


In [3]:
# Parameters

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

cost = {
    ('Jan', 'VEG1'): 110,
    ('Jan', 'VEG2'): 120,
    ('Jan', 'OIL1'): 130,
    ('Jan', 'OIL2'): 110,
    ('Jan', 'OIL3'): 115,
    ('Feb', 'VEG1'): 130,
    ('Feb', 'VEG2'): 130,
    ('Feb', 'OIL1'): 110,
    ('Feb', 'OIL2'): 90,
    ('Feb', 'OIL3'): 115,
    ('Mar', 'VEG1'): 110,
    ('Mar', 'VEG2'): 140,
    ('Mar', 'OIL1'): 130,
    ('Mar', 'OIL2'): 100,
    ('Mar', 'OIL3'): 95,
    ('Apr', 'VEG1'): 120,
    ('Apr', 'VEG2'): 110,
    ('Apr', 'OIL1'): 120,
    ('Apr', 'OIL2'): 120,
    ('Apr', 'OIL3'): 125,
    ('May', 'VEG1'): 100,
    ('May', 'VEG2'): 120,
    ('May', 'OIL1'): 150,
    ('May', 'OIL2'): 110,
    ('May', 'OIL3'): 105,
    ('Jun', 'VEG1'): 90,
    ('Jun', 'VEG2'): 100,
    ('Jun', 'OIL1'): 140,
    ('Jun', 'OIL2'): 80,
    ('Jun', 'OIL3'): 135
}


hardness = {"VEG1": 8.8, "VEG2": 6.1, "OIL1": 2.0, "OIL2": 4.2, "OIL3": 5.0}

price = 150
init_store = 500
target_store = 500
veg_cap = 200
oil_cap = 250

min_hardness = 3
max_hardness = 6
holding_cost = 5

In [4]:
# Model deployment

food = gp.Model('Food Manufacture I')
# Quantity of food produced in each period
produce = food.addVars(months, name="Produce")
# Quantity bought of each product in each period
buy = food.addVars(months, oils, name = "Buy")
# Quantity used of each product  in each period
consume = food.addVars(months, oils, name = "Use")
# Quantity stored of each product  in each period
store = food.addVars(months, oils, name = "Store")



Using license file /home/dreth/gurobi.lic
Academic license - for non-commercial use only


In [5]:
#1. Initial Balance
Balance0 = food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")

#2. Balance
Balance = food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != month[0]), "Balance")

#3. Inventory Target
TargetInv = food.addConstrs((store[months[-1], oil] == target_store for oil in oils),"End_Balance")

#4.1 Vegetable Oil Capacity
VegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")

#4.2 Non-vegetable Oil Capacity
NonVegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")

#5. Hardness
HardnessMin = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")
HardnessMax = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")

#6. Mass Conservation
MassConservation = food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")

#0. Objective Function
obj = price*produce.sum() - buy.prod(cost) - holding_cost*store.sum()
food.setObjective(obj, GRB.MAXIMIZE) # maximize profit

In [6]:
food.optimize()



Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 70 rows, 96 columns and 278 nonzeros
Model fingerprint: 0xd588eb19
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+02]
Presolve removed 33 rows and 45 columns
Presolve time: 0.00s
Presolved: 37 rows, 51 columns, 149 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7375000e+05   1.703125e+03   0.000000e+00      0s
      32    1.0784259e+05   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.00 seconds
Optimal objective  1.078425926e+05


In [56]:
# Variable information including sensitivity information
for n,v in enumerate(food.getVars()):
    if n>1 and food.getVars()[n-1].Varname[0:3] != v.Varname[0:3]:
        print('----------------------------------------------')
    print(f"{v.Varname} = {v.X}, reduced cost = {abs(v.RC):.2f}, from coeff = {v.SAObjLow:.2f}, to coeff = {v.SAObjUp:.2f}")
    



Produce[Jan] = 450.0, reduced cost = 0.00, from coeff = 85.00, to coeff = inf
Produce[Feb] = 450.0, reduced cost = 0.00, from coeff = 90.00, to coeff = inf
Produce[Mar] = 450.0, reduced cost = 0.00, from coeff = 95.00, to coeff = inf
Produce[Apr] = 450.0, reduced cost = 0.00, from coeff = 100.00, to coeff = inf
Produce[May] = 450.0, reduced cost = 0.00, from coeff = 105.00, to coeff = inf
Produce[Jun] = 450.0, reduced cost = 0.00, from coeff = 100.37, to coeff = inf
----------------------------------------------
Buy[Jan,VEG1] = 0.0, reduced cost = 35.00, from coeff = -inf, to coeff = -75.00
Buy[Jan,VEG2] = 0.0, reduced cost = 45.00, from coeff = -inf, to coeff = -75.00
Buy[Jan,OIL1] = 0.0, reduced cost = 45.00, from coeff = -inf, to coeff = -85.00
Buy[Jan,OIL2] = 0.0, reduced cost = 25.00, from coeff = -inf, to coeff = -85.00
Buy[Jan,OIL3] = 0.0, reduced cost = 30.00, from coeff = -inf, to coeff = -85.00
Buy[Feb,VEG1] = 0.0, reduced cost = 50.00, from coeff = -inf, to coeff = -80.00
Bu

In [17]:
# Optimal objective value
print(f"{food.objVal:.3f}")

107842.593


In [58]:
# Optimal shadow prices
for n,c in enumerate(food.getConstrs()):
    if n>1 and food.getConstrs()[n-1].ConstrName[0:3] != c.ConstrName[0:3]:
        print('----------------------------------------------')
    print(f"{c.ConstrName} : shadow price = {c.Pi}, from RHS = {c.SARHSLow}, to RHS = {c.SARHSUp:.2f}")

Initial_Balance[VEG1] : shadow price = 0.0, from RHS = -500.0, to RHS = -500.00
Initial_Balance[VEG2] : shadow price = 0.0, from RHS = -500.0, to RHS = -500.00
Initial_Balance[OIL1] : shadow price = 0.0, from RHS = -500.0, to RHS = -500.00
Initial_Balance[OIL2] : shadow price = 0.0, from RHS = -500.0, to RHS = -500.00
Initial_Balance[OIL3] : shadow price = 0.0, from RHS = -500.0, to RHS = -500.00
----------------------------------------------
Balance[VEG1,Jan] : shadow price = -75.0, from RHS = 0.0, to RHS = 0.00
Balance[VEG1,Feb] : shadow price = -80.0, from RHS = -62.96296296296294, to RHS = 0.00
Balance[VEG1,Mar] : shadow price = -85.0, from RHS = -62.96296296296294, to RHS = 0.00
Balance[VEG1,Apr] : shadow price = -90.0, from RHS = -62.96296296296294, to RHS = 0.00
Balance[VEG1,May] : shadow price = -95.0, from RHS = -62.96296296296294, to RHS = 0.00
Balance[VEG1,Jun] : shadow price = -90.0, from RHS = -659.2592592592592, to RHS = inf
Balance[VEG2,Jan] : shadow price = -75.0, from 

<img src="./images/prob_3/b.png"/>

<img src="./images/prob_3/c.png"/>